In [1]:
!pip install eli5

     |████████████████████████████████| 112kB 2.9MB/s 


In [0]:
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor

from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import cross_val_score
import eli5
from eli5.sklearn import PermutationImportance

from ast import literal_eval
from tqdm import tqdm_notebook

In [3]:
cd /content/drive/My\ Drive/Colab\ Notebooks/dw_matrix


/content/drive/My Drive/Colab Notebooks/dw_matrix


In [4]:
ls data

men_shoes.csv  shoes_prices.csv


In [0]:
df = pd.read_csv("data/men_shoes.csv", low_memory=False)

In [0]:
def run_model(feats, model = DecisionTreeRegressor(max_depth=5)):
  X = df[feats].values
  y = df.prices_amountmin.values

  scores = cross_val_score(model, X, y, scoring="neg_mean_absolute_error")
  return np.mean(scores), np.std(scores)

In [11]:
df['brand_cat'] = df['brand'].map(lambda x: str(x).lower()).factorize()[0]
run_model(['brand_cat'])

(-58.133398968282776, 4.206122611474276)

In [12]:
model = RandomForestRegressor(max_depth=5, n_estimators=100, random_state=0)
run_model(['brand_cat'], model)

(-57.31783843165656, 4.181246596160967)

In [0]:
def parse_features(x):
  if str(x) == 'nan': return {}
  output_dict = {}
  features = literal_eval(x.replace('\\"', '"'))
  for item in features:
    key= item['key'].lower().strip()
    value = item['value'][0].lower().strip()
    output_dict[key] = value
  return output_dict

df['features_parsed'] = df['features'].map(parse_features)

In [20]:
df.features_parsed.head().values

array([{'gender': 'men', 'shoe size': 'm', 'shoe category': "men's shoes", 'color': 'multicolor', 'manufacturer part number': '8190-w-navy-7.5', 'brand': 'josmo'},
       {'gender': 'men', 'shoe size': 'm', 'shoe category': "men's shoes", 'color': 'multicolor', 'manufacturer part number': '8190-w-navy-7.5', 'brand': 'josmo'},
       {'gender': 'men', 'color': 'black', 'shipping weight (in pounds)': '0.45', 'condition': 'new', 'brand': 'servus by honeywell', 'manufacturer_part_number': 'zsr101blmlg'},
       {'gender': 'men', 'color': 'black', 'shipping weight (in pounds)': '0.45', 'condition': 'new', 'brand': 'servus by honeywell', 'manufacturer_part_number': 'zsr101blmlg'},
       {'gender': 'men', 'color': 'black', 'shipping weight (in pounds)': '0.45', 'condition': 'new', 'brand': 'servus by honeywell', 'manufacturer_part_number': 'zsr101blmlg'}],
      dtype=object)

In [24]:
keys = set()
df['features_parsed'].map(lambda x: keys.update(x.keys()))
len(keys)

476

In [25]:
def get_name_feat(key):
  return 'feat_' + key
for key in tqdm_notebook(keys):
  df[get_name_feat(key)] = df.features_parsed.map(lambda feats: feats[key] if key in feats else np.nan)

In [0]:
keys_stat = {}
for key in keys:
  y = df[get_name_feat(key)].isnull()
  z = df[ False == y].shape[0] / df.shape[0] * 100
  keys_stat[key] = z

In [37]:
{k:v for k,v in keys_stat.items() if v > 30}

{'brand': 48.62691466083151,
 'color': 47.784463894967175,
 'gender': 50.17505470459519,
 'manufacturer part number': 36.252735229759296,
 'material': 34.9070021881838}

In [0]:
df['feat_brand_cat'] = df['feat_brand'].factorize()[0]
df['feat_color_cat'] = df['feat_color'].factorize()[0]
df['feat_gender_cat'] = df['feat_gender'].factorize()[0]
df['feat_manufacturer_cat'] = df['feat_manufacturer'].factorize()[0]
df['feat_material_cat'] = df['feat_material'].factorize()[0]

df['feat_sport_cat'] = df['feat_sport'].factorize()[0]
df['feat_style_cat'] = df['feat_style'].factorize()[0]

for key in keys:
  df[get_name_feat(key) + "_cat"] = df[get_name_feat(key)].factorize()[0]

In [0]:
df['brand'] = df['brand'].map(lambda x: str(x).lower())

In [67]:
feats_cat = [x for x in df.columns if x.endswith("cat")]
feats_cat

['manufacturer_cat',
 'brand_cat',
 'feat_brand_cat',
 'feat_color_cat',
 'feat_gender_cat',
 'feat_manufacturer_cat',
 'feat_material_cat',
 'feat_sport_cat',
 'feat_style_cat',
 'feat_fabric care instructions_cat',
 'feat_ground_cat',
 'feat_color mapping_cat',
 'feat_animal type_cat',
 'feat_leather grade_cat',
 'feat_footwear type_cat',
 'feat_assembled in country of origin_cat',
 'feat_tactical_cat',
 'feat_character_cat',
 'feat_toe type_cat',
 'feat_fuel type_cat',
 'feat_feature_cat',
 'feat_inv_cat',
 'feat_is lined_cat',
 'feat_full product manual_cat',
 'feat_product id_cat',
 'feat_item style_cat',
 'feat_terrain_cat',
 'feat_age segment_cat',
 'feat_clothing type_cat',
 'feat_stylenumber_cat',
 'feat_primary color_cat',
 'feat_case material_cat',
 'feat_case type_cat',
 'feat_resolution_cat',
 'feat_casual & dress shoe style_cat',
 'feat_capacity_cat',
 'feat_hardsided or softsided_cat',
 'feat_authenticity_cat',
 'feat_lens material:_cat',
 'feat_main color_cat',
 'feat_c

In [0]:
feats = ['brand_cat', 'feat_brand_cat', 'feat_gender_cat', 'feat_material_cat', 'feat_sport_cat', 'feat_style_cat']
feats += ['feat_metal type_cat', 'feat_occasion_cat', 'feat_shape_cat']
#feats += feats_cat
#feats = list(set(feats))

In [78]:
model = RandomForestRegressor(max_depth=5, n_estimators=100)
result = run_model(feats, model)
result

(-57.23801651202255, 4.296396092033771)

In [79]:
X = df[feats].values
y = df.prices_amountmin.values
m = RandomForestRegressor(max_depth=5, n_estimators=100, random_state=0)
m.fit(X, y)

print(result)
perm = PermutationImportance(m, random_state=1).fit(X, y)
eli5.show_weights(perm, feature_names=feats)

(-57.23801651202255, 4.296396092033771)


Weight,Feature
0.2592 ± 0.0106,brand_cat
0.1027 ± 0.0029,feat_material_cat
0.0229 ± 0.0029,feat_gender_cat
0.0172 ± 0.0004,feat_brand_cat
0.0123 ± 0.0003,feat_shape_cat
0.0083 ± 0.0020,feat_metal type_cat
0.0036 ± 0.0008,feat_occasion_cat
0.0021 ± 0.0007,feat_style_cat
0.0002 ± 0.0000,feat_sport_cat


nike                    1777
puma                     609
ralph lauren             526
vans                     386
new balance              371
                        ... 
lanbaosi                   1
iecool                     1
thrive camp tactical       1
louis m gerson             1
sammons                    1
Name: brand, Length: 1732, dtype: int64